In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Multifactor model for pricing Closed-End Funds
Hristo T. Raykov  
email: hristocr@gmail.com

## Abstract

## Keywords

## 1. Introduction
Problem Statement

## 2. Literature Review
Previous research

## 3. Methods
Gathering / generating, cleaning, and formatting data. How the research was done. Form a hypothesis.  
Data  
Regression  
etc...

## 4. Results 
Testing and visualization. Report what you actually found. Test the hypothesis with an experiment.

## 5. Discussion
Interpret your findings for the reader in relation to previous research and the literature as a whole.

## 6. Conclusion
Communication. 

## References


